In [ ]:
from typing import Literal, TypedDict, Union
import json

Starfish = TypedDict("Starfish", {
    "star_shaped": Literal[True],
    "fins": Literal[0],
    "name": str,
    "on a rock": bool
})

Jellyfish = TypedDict("Jellyfish", {
    "star_shaped": Literal[False],
    "fins": Literal[None],
    "name": str
})

Fish = TypedDict("Fish", {
    "star_shaped": Literal[False],
    "fins": int,
    "name": str
})

# Here, the `type | type` syntax could be used, but in the case of a tagged union it is more semantically explicit to wrap them as a singular unit
SeaCreature = Union[Starfish, Jellyfish, Fish]

json_obj = '''
{
    ...unknown
}
'''

sea_creature: SeaCreature = json.loads(json_obj)

if sea_creature["star_shaped"] == True: #! Note that if you wrote `if sea_creature["star_shaped"]: ` it wouldn't work!!!
    fins = sea_creature["fins"] # Known as `Literal[0]` 👍

if sea_creature["star_shaped"] == False:
    fins = sea_creature['fins'] # Known as `int | None`, no suggestion for 'on a rock' 👍